#Accuracy : 88.76%


In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from skimage import transform
import numpy as np
from keras.callbacks import ModelCheckpoint, CSVLogger
import os as os
from google.colab import files
import skimage

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 32
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 2
dropout_rate = 0.2

In [50]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

X_train = []
X_test = []
for i in range(0, x_train.shape[0]):
  X_train.append(skimage.transform.resize(x_train[i],(24,24)))

X_train = np.asarray(X_train)   
  
for i in range(0, x_test.shape[0]):
  X_test.append(skimage.transform.resize(x_test[i],(24,24)))

X_test = np.asarray(X_test)  

img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

X_train = X_train.reshape(X_train.shape[0], img_height, img_width, channel)
X_test = X_test.reshape(X_test.shape[0], img_height, img_width, channel)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255 # Normalise data to [0, 1] range
X_test /= 255 # Normalise data to [0, 1] range

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
       
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
       
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
   
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12  

input = Input(shape=(img_height, img_width, channel,))
#Conv2D_1_1 = Conv2D(num_filter, (1,1), use_bias=False)(input)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False)(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)
  
Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [55]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 24, 24, 3)    0                                            
__________________________________________________________________________________________________
conv2d_157 (Conv2D)             (None, 22, 22, 12)   324         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_157 (BatchN (None, 22, 22, 12)   48          conv2d_157[0][0]                 
__________________________________________________________________________________________________
activation_157 (Activation)     (None, 22, 22, 12)   0           batch_normalization_157[0][0]    
__________________________________________________________________________________________________
conv2d_158

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
logger = CSVLogger(os.path.join("LogFile.log"))
callbacks_list = [checkpoint, logger]

In [58]:
model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=15,
                    verbose=1,
                    validation_data=(X_test, y_test), 
                    callbacks = callbacks_list,
                    shuffle='True')

Train on 50000 samples, validate on 10000 samples
Epoch 1/15
21856/50000 [============>.................] - ETA: 5:29 - loss: 1.8016 - acc: 0.3353

50000/50000 [==============================] - 590s 12ms/step - loss: 1.5992 - acc: 0.4111 - val_loss: 1.5349 - val_acc: 0.4647

Epoch 00001: val_acc improved from -inf to 0.46470, saving model to weights.best.hdf5
Epoch 2/15
 2528/50000 [>.............................] - ETA: 8:59 - loss: 1.3120 - acc: 0.5269

49984/50000 [============================>.] - ETA: 0s - loss: 1.1664 - acc: 0.5825

50000/50000 [==============================] - 577s 12ms/step - loss: 1.1665 - acc: 0.5825 - val_loss: 1.3455 - val_acc: 0.5571

Epoch 00002: val_acc improved from 0.46470 to 0.55710, saving model to weights.best.hdf5
Epoch 3/15
 7968/50000 [===>..........................] - ETA: 7:40 - loss: 1.0050 - acc: 0.6344

49984/50000 [============================>.] - ETA: 0s - loss: 0.9540 - acc: 0.6605

...

50000/50000 [==============================] - 578s 12ms/step - loss: 0.4277 - acc: 0.8508 - val_loss: 0.6744 - val_acc: 0.7917

Epoch 00013: val_acc did not improve from 0.80210
Epoch 14/15
 8832/50000 [====>.........................] - ETA: 7:35 - loss: 0.3653 - acc: 0.8740

49984/50000 [============================>.] - ETA: 0s - loss: 0.3965 - acc: 0.8620

50000/50000 [==============================] - 579s 12ms/step - loss: 0.3966 - acc: 0.8620 - val_loss: 0.6623 - val_acc: 0.7999

Epoch 00014: val_acc did not improve from 0.80210
Epoch 15/15
 8832/50000 [====>.........................] - ETA: 7:36 - loss: 0.3696 - acc: 0.8696

49984/50000 [============================>.] - ETA: 0s - loss: 0.3790 - acc: 0.8683

50000/50000 [==============================] - 578s 12ms/step - loss: 0.3790 - acc: 0.8683 - val_loss: 0.5543 - val_acc: 0.8285

Epoch 00015: val_acc improved from 0.80210 to 0.82850, saving model to weights.best.hdf5


In [0]:
# Save the trained weights in to .h5 format
model.save_weights('model_weights.h5')

In [0]:
files.download('model_weights.h5')
files.download('LogFile.log')
files.download('weights.best.hdf5')

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12  

img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

x_train = x_train.reshape(x_train.shape[0], img_height, img_width, channel)
x_test = x_test.reshape(x_test.shape[0], img_height, img_width, channel)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 # Normalise data to [0, 1] range
x_test /= 255 # Normalise data to [0, 1] range

input1 = Input(shape=(img_height, img_width, channel,))

First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False)(input1)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)
  
Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output1 = output_layer(Last_Block)

In [62]:
model1 = Model(inputs=[input1], outputs=[output1])
model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_209 (Conv2D)             (None, 30, 30, 12)   324         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_209 (BatchN (None, 30, 30, 12)   48          conv2d_209[0][0]                 
__________________________________________________________________________________________________
activation_209 (Activation)     (None, 30, 30, 12)   0           batch_normalization_209[0][0]    
__________________________________________________________________________________________________
conv2d_210

In [0]:
# determine Loss function and Optimizer
model1.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

model1.load_weights('model_weights.h5')

In [64]:
# checkpoint
filepath="weights.best2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
logger = CSVLogger(os.path.join("LogFile2.log"))
callbacks_list1 = [checkpoint, logger]

model1.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=35,
                    verbose=1,
                    validation_data=(x_test, y_test), 
                    callbacks = callbacks_list1,
                    shuffle='True')

Train on 50000 samples, validate on 10000 samples
Epoch 1/35
21856/50000 [============>.................] - ETA: 5:26 - loss: 0.5136 - acc: 0.8230

50000/50000 [==============================] - 594s 12ms/step - loss: 0.4682 - acc: 0.8384 - val_loss: 0.4719 - val_acc: 0.8418

Epoch 00001: val_acc improved from -inf to 0.84180, saving model to weights.best2.hdf5
Epoch 2/35
 2528/50000 [>.............................] - ETA: 8:43 - loss: 0.3851 - acc: 0.8687

49984/50000 [============================>.] - ETA: 0s - loss: 0.3729 - acc: 0.8711

50000/50000 [==============================] - 578s 12ms/step - loss: 0.3729 - acc: 0.8711 - val_loss: 0.4918 - val_acc: 0.8442

Epoch 00002: val_acc improved from 0.84180 to 0.84420, saving model to weights.best2.hdf5
Epoch 3/35
 7936/50000 [===>..........................] - ETA: 7:40 - loss: 0.3279 - acc: 0.8880

49984/50000 [============================>.] - ETA: 0s - loss: 0.3341 - acc: 0.8828

...

50000/50000 [==============================] - 563s 11ms/step - loss: 0.0798 - acc: 0.9723 - val_loss: 0.5172 - val_acc: 0.8832

Epoch 00033: val_acc did not improve from 0.88580
Epoch 34/35
 8832/50000 [====>.........................] - ETA: 7:18 - loss: 0.0739 - acc: 0.9757

49984/50000 [============================>.] - ETA: 0s - loss: 0.0771 - acc: 0.9727

50000/50000 [==============================] - 559s 11ms/step - loss: 0.0771 - acc: 0.9727 - val_loss: 0.5119 - val_acc: 0.8833

Epoch 00034: val_acc did not improve from 0.88580
Epoch 35/35
 8832/50000 [====>.........................] - ETA: 7:16 - loss: 0.0742 - acc: 0.9727

49984/50000 [============================>.] - ETA: 0s - loss: 0.0766 - acc: 0.9716

50000/50000 [==============================] - 561s 11ms/step - loss: 0.0766 - acc: 0.9716 - val_loss: 0.5356 - val_acc: 0.8876

Epoch 00035: val_acc improved from 0.88580 to 0.88760, saving model to weights.best2.hdf5


In [0]:
# Save the trained weights in to .h5 format
model.save_weights('model_weights32.h5')

files.download('model_weights32.h5')
files.download('LogFile2.log')
files.download('weights.best2.hdf5')



In [0]:
model1.load_weights('weights.best2.hdf5')

In [69]:
# Test the model
score = model1.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 30s 3ms/step
Test loss: 0.5355630850374699
Test accuracy: 0.8876
